<a href="https://colab.research.google.com/github/yfhkucing/latihan_dicoding/blob/master/CatDog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#library yang akan dipakai
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline
from google.colab import files
from keras import optimizers
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#versi tensorflow yang digunakan
print(tf.__version__)

2.9.2


In [6]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets download tongpython/cat-and-dog

100% 217M/218M [00:06<00:00, 35.2MB/s]
100% 218M/218M [00:06<00:00, 34.9MB/s]


In [ ]:
! unzip /content/cat-and-dog.zip

In [ ]:
#menghitung jumlah data
cat = os.listdir('/content/training_set/training_set/cats')
dog = os.listdir('/content/training_set/training_set/dogs')
print("data cat : "+str(len(cat)))
print("data dog : "+str(len(dog)))

In [19]:
#split data untuk training dan validation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
dir = '/content/training_set/training_set'

'''data di split sebanyak 40% untuk validation, augmentasi berupa 
rescale, rotation, horizontal flip, dan shear range'''

Image_Generator = ImageDataGenerator( 
                      validation_split = 0.4,
                      rescale=1./255,
                      rotation_range=20,
                      horizontal_flip=True,
                      shear_range = 0.2)

Training_Data = Image_Generator.flow_from_directory(dir,
                                                    target_size=(150,150),
                                                    batch_size=4,
                                                    shuffle=True,
                                                    class_mode='binary',
                                                    subset='training')
Validation_Data = Image_Generator.flow_from_directory(dir,
                                                      target_size=(150,150),
                                                      batch_size=4,
                                                      shuffle=True,
                                                      class_mode='binary',
                                                      subset='validation')

Found 4803 images belonging to 2 classes.
Found 3202 images belonging to 2 classes.


In [21]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [22]:
'''konfigurasi model untuk training. Optimizer menggunakan Adam, loss function menggunakan
categorical crossentropy, dan matriks untuk pengujian menggnakan matriks akurasi'''

model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 17, 17, 128)     

In [24]:
#training model
history = model.fit(
    Training_Data, 
    validation_data  = Validation_Data,
    epochs = 50,
    steps_per_epoch = 30,
    verbose=2)

Epoch 1/50
30/30 - 25s - loss: 0.7221 - accuracy: 0.4833 - val_loss: 0.6935 - val_accuracy: 0.4997 - 25s/epoch - 843ms/step
Epoch 2/50
30/30 - 25s - loss: 0.6960 - accuracy: 0.5000 - val_loss: 0.6930 - val_accuracy: 0.4997 - 25s/epoch - 824ms/step
Epoch 3/50
30/30 - 25s - loss: 0.6933 - accuracy: 0.5333 - val_loss: 0.6929 - val_accuracy: 0.5003 - 25s/epoch - 830ms/step
Epoch 4/50
30/30 - 24s - loss: 0.6944 - accuracy: 0.4667 - val_loss: 0.6931 - val_accuracy: 0.5003 - 24s/epoch - 800ms/step
Epoch 5/50
30/30 - 24s - loss: 0.6934 - accuracy: 0.4250 - val_loss: 0.6931 - val_accuracy: 0.6118 - 24s/epoch - 809ms/step
Epoch 6/50
30/30 - 25s - loss: 0.6936 - accuracy: 0.6083 - val_loss: 0.7014 - val_accuracy: 0.4997 - 25s/epoch - 841ms/step
Epoch 7/50
30/30 - 24s - loss: 0.7030 - accuracy: 0.5250 - val_loss: 0.6934 - val_accuracy: 0.5003 - 24s/epoch - 810ms/step
Epoch 8/50
30/30 - 24s - loss: 0.6969 - accuracy: 0.4750 - val_loss: 0.6931 - val_accuracy: 0.5003 - 24s/epoch - 814ms/step
Epoch 9/

KeyboardInterrupt: ignored

In [ ]:
Akurasi_train = history.history['accuracy']
Akurasi_val = history.history['val_accuracy']

epochs = range(len(Akurasi_train))

plt.plot(epochs, Akurasi_train, 'b', label='Akurasi Training')
plt.plot(epochs, Akurasi_val, 'y', label='Akurasi Validasi')
plt.title('Akurasi Training dan Validasi')
plt.legend(loc=0)
plt.show()